In [1]:
import torch
import pickle
from torch import nn,optim
import torch.nn.functional
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os as os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,multilabel_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from scipy.stats import kurtosis,skew
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import scipy.stats
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from imblearn.over_sampling import SMOTE


from sklearn.model_selection import cross_val_score
from collections import OrderedDict

#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)


<h1>Train data prep</h1>

una osservazione ogni secondo <br>
1200 osservazioni per dataframe<br>
Le colonne P1-P7 sono pressioni nei punti di misurazione<br>

In [2]:
path_dataset= '/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataset'
path_dataframes="/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataframes"
path_data = "/data"
path_labels = "/labels_spacecraft.xlsx"

def create_dataframe(name):
    dfs_data = []
    filenames = sorted(os.listdir(f"{path_dataset}/{name}/data"))
    for case,filename in enumerate(filenames):
        if filename.endswith('.csv'):
            percorso_file = os.path.join(f"{path_dataset}/{name}/data", filename)
            df = pd.read_csv(percorso_file)
            print(filename)
            df["Case"]=case+1
            
            dfs_data.append(df)

    df_data = pd.concat(dfs_data, ignore_index=True)
    return df_data,dfs_data

def create_df_labels(name,filename):
    df = pd.read_excel(f"{path_dataset}/{name}/{filename}.xlsx",header=1)
    if name == "train":
        df = df.rename(columns={"Unnamed: 0":"Case","Unnamed: 1":"Spacecraft","Unnamed: 2":"Condition"})
    elif name == "test":
        df = df.rename(columns={"Case#":"Case","Spacecraft#":"Spacecraft"})
        
    return df


In [ ]:
def map_condition(value):
    if value == 1:
        return 0
    elif value in [2, 3]:
        return 1
    else:
        return None  

In [3]:
def prepare_df_labels(df_labels):
    condition_map = {'Normal':1,'Fault':2,'Anomaly':3}
    bp_map = {'No':0,'Yes':1}
    df_labels["Condition"] = df_labels["Condition"].map(condition_map)
    bp_columns = [col for col in df_labels.columns if ("BP" in col) or("BV" in col)]

 

    df_labels['binary_condition'] = df_labels['Condition'].apply(map_condition)

    for col in bp_columns:
        df_labels[col] = df_labels[col].map(bp_map)
        
    return df_labels


In [4]:
df_train,dfs_train = create_dataframe("train")
df_test,dfs_test = create_dataframe("test")

df_labels_train = create_df_labels("train","labels")
df_labels_test = create_df_labels("test","labels_spacecraft")
df_labels_train = prepare_df_labels(df_labels_train)

Case001.csv
Case002.csv
Case003.csv
Case004.csv
Case005.csv
Case006.csv
Case007.csv
Case008.csv
Case009.csv
Case010.csv
Case011.csv
Case012.csv
Case013.csv
Case014.csv
Case015.csv
Case016.csv
Case017.csv
Case018.csv
Case019.csv
Case020.csv
Case021.csv
Case022.csv
Case023.csv
Case024.csv
Case025.csv
Case026.csv
Case027.csv
Case028.csv
Case029.csv
Case030.csv
Case031.csv
Case032.csv
Case033.csv
Case034.csv
Case035.csv
Case036.csv
Case037.csv
Case038.csv
Case039.csv
Case040.csv
Case041.csv
Case042.csv
Case043.csv
Case044.csv
Case045.csv
Case046.csv
Case047.csv
Case048.csv
Case049.csv
Case050.csv
Case051.csv
Case052.csv
Case053.csv
Case054.csv
Case055.csv
Case056.csv
Case057.csv
Case058.csv
Case059.csv
Case060.csv
Case061.csv
Case062.csv
Case063.csv
Case064.csv
Case065.csv
Case066.csv
Case067.csv
Case068.csv
Case069.csv
Case070.csv
Case071.csv
Case072.csv
Case073.csv
Case074.csv
Case075.csv
Case076.csv
Case077.csv
Case078.csv
Case079.csv
Case080.csv
Case081.csv
Case082.csv
Case083.csv
Case

In [5]:
df_test

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1
...,...,...,...,...,...,...,...,...,...
55241,1.196,2.017816,2.016297,2.012680,2.044149,2.037457,2.033628,2.069056,46
55242,1.197,2.039878,2.055107,2.054626,2.071298,2.072899,2.076782,2.093548,46
55243,1.198,2.054024,2.081678,2.094493,2.096693,2.095300,2.107353,2.138567,46
55244,1.199,2.057180,2.094954,2.097106,2.153193,2.099423,2.112326,2.127471,46


In [6]:
df_labels_train.head()

,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition
0,1,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
1,2,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
2,3,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
3,4,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0
4,5,1,1,100,100,100,100,0,0,0,0,0,0,0,0,0


In [7]:
with open('dfs_train_list.pkl', 'wb') as file:
    pickle.dump(dfs_train, file)

with open('dfs_test_list.pkl', 'wb') as file:
    pickle.dump(dfs_test, file)



df_labels_train.to_csv(f"{path_dataframes}/df_labels_train.csv",index_label="index")
df_labels_test.to_csv(f"{path_dataframes}/df_labels_test.csv",index_label="index")
# df_train.to_csv(f"{path_dataframes}/df_train")